<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [2]</a>'.</span>

# Exploratory Data Analysis - LendingClub Dataset

## Project: Alternative Credit Scoring Platform

This notebook performs initial exploratory data analysis on the LendingClub dataset to understand the features, distributions, and data quality issues. The dataset contains information about loans issued between 2007 and 2018, with a target variable indicating whether a loan defaulted or not.

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Set up plotting styles
plt.style.use('default')
sns.set_palette('husl')
warnings.filterwarnings('ignore')

# Set plotting options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

## Load the Dataset

Let's load the LendingClub dataset and examine its basic structure.

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [2]:
# Load the dataset
df = pd.read_csv('../data/raw/LC_loans_granting_model_dataset.csv')

# Display basic information about the dataset
print('Dataset shape:', df.shape)
print('\nDataset info:')
df.info()

FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/LC_loans_granting_model_dataset.csv'

In [ ]:
# Display the first few rows
df.head()

In [ ]:
# Display basic statistics
df.describe()

## Dataset Overview

Based on the information from Zenodo, this dataset has been specifically prepared for granting models that make decisions on loan approval based on information available at application time. It includes loans with final status (either "Fully Paid" or "Default") to avoid data leakage.

**Target Variable**: Default (binary)
- 0 = Fully Paid
- 1 = Default (charged off)

### Features

1. **Loan Identification**: `id`, `issue_d`
2. **Quantitative Variables**: `revenue`, `dti_n`, `loan_amnt`, `fico_n`, `experience_c`
3. **Categorical Variables**: `emp_length`, `purpose`, `home_ownership_n`, `addr_state`, `zip_code`
4. **Textual Variables**: `title`, `desc`

## Target Variable Analysis

Let's examine the distribution of the target variable to understand the class balance.

In [ ]:
# Analyze the target variable distribution
target_counts = df['Default'].value_counts()
target_percentages = df['Default'].value_counts(normalize=True) * 100

print('Target variable distribution:')
print(target_counts)
print('\nTarget variable percentages:')
print(target_percentages)

# Visualize the target distribution
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.pie(target_counts.values, labels=['Fully Paid (0)', 'Default (1)'], autopct='%1.1f%%', startangle=90)
plt.title('Distribution of Target Variable')

plt.subplot(1, 2, 2)
sns.countplot(data=df, x='Default')
plt.title('Count of Each Target Class')
plt.xlabel('Default Status (0=Fully Paid, 1=Default)')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

## Missing Values Analysis

Let's check for missing values in the dataset.

In [ ]:
# Check for missing values
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100

missing_df = pd.DataFrame({
    'Missing Count': missing_values,
    'Missing Percentage': missing_percentage
}).sort_values(by='Missing Percentage', ascending=False)

print('Features with missing values (top 10):')
print(missing_df[missing_df['Missing Count'] > 0].head(10))

## Numerical Features Analysis

Let's examine the distributions of numerical features and their relationship with the target variable.

In [ ]:
# Identify numerical columns
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
numerical_cols.remove('Default')  # Remove target variable from numerical columns

print('Numerical features:', numerical_cols)

In [ ]:
# Distribution plots for numerical features
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

for i, col in enumerate(numerical_cols):
    if i >= len(axes):
        break
        
    # Plot distribution by target
    sns.histplot(data=df, x=col, hue='Default', kde=True, ax=axes[i])
    axes[i].set_title(f'Distribution of {col} by Default Status')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Count')

# Hide unused subplots
for i in range(len(numerical_cols), len(axes)):
    axes[i].set_visible(False)

plt.tight_layout()
plt.show()

In [ ]:
# Statistical summary of numerical features by target
numerical_stats = df.groupby('Default')[numerical_cols].describe()
print('Statistical summary by Default status:')
numerical_stats

## Categorical Features Analysis

Let's examine the categorical features and their relationship with the target variable.

In [ ]:
# Identify categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

print('Categorical features:', categorical_cols)

In [ ]:
# Analyze categorical features
for col in categorical_cols:
    print(f'\nFeature: {col}')
    print(f'Unique values: {df[col].nunique()}')
    print('Top 10 values:')
    print(df[col].value_counts().head(10))
    
    # Create a cross-tabulation with the target variable
    crosstab = pd.crosstab(df[col], df['Default'], normalize='index') * 100
    print('\nPercentage of defaults by category (top 10):')
    print(crosstab.sort_values(by=1, ascending=False).head(10))
    print('-' * 50)

In [ ]:
# Visualize categorical features vs target
fig, axes = plt.subplots(2, 2, figsize=(20, 12))
axes = axes.ravel()

for i, col in enumerate(categorical_cols[:4]):  # Show top 4 categorical features
    # Calculate default rate by category
    default_rate = df.groupby(col)['Default'].mean().sort_values(ascending=False)
    
    # Create bar plot
    sns.barplot(x=default_rate.index[:10], y=default_rate.values[:10], ax=axes[i])
    axes[i].set_title(f'Default Rate by {col} (Top 10 Categories)')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Default Rate')
    axes[i].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## Feature Correlations

Let's examine correlations between numerical features to identify potential multicollinearity issues.

In [ ]:
# Calculate correlation matrix
correlation_matrix = df[numerical_cols].corr()

# Create a heatmap of the correlation matrix
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, fmt='.2f', cbar_kws={'shrink': 0.8})
plt.title('Correlation Matrix of Numerical Features')
plt.tight_layout()
plt.show()

## Data Quality Issues

Let's check for potential data quality issues such as outliers, inconsistent values, etc.

In [ ]:
# Check for outliers in numerical features using IQR method
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers

# Check outliers for each numerical feature
for col in numerical_cols:
    outliers = detect_outliers_iqr(df, col)
    print(f'{col}: {len(outliers)} outliers ({len(outliers)/len(df)*100:.2f}% of data)')

## Key Findings and Hypotheses

### Initial Findings:
1. **Dataset Size**: The dataset contains [fill in number] records with [fill in number] features
2. **Target Distribution**: The dataset is [balanced/imbalanced] with [X]% of loans defaulting
3. **Missing Values**: [State any significant missing value patterns]
4. **Feature Relationships**: [Note any obvious patterns between features and target]

### Hypotheses:
1. **FICO Score**: Lower FICO scores are likely associated with higher default rates
2. **Debt-to-Income Ratio**: Higher DTI ratios may correlate with increased default risk
3. **Loan Amount**: The relationship between loan amount and default risk may be complex
4. **Employment Length**: Shorter employment history might indicate higher risk
5. **Purpose**: Loan purpose may affect default rates (e.g. debt consolidation vs. small business)

### Data Quality Notes:
- [Note any data quality issues identified]
- [Note any potential preprocessing steps needed]

### Next Steps:
1. Perform feature engineering to create additional predictive features
2. Address missing values and outliers
3. Prepare data for modeling
4. Develop baseline models for comparison